In [1]:
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
import re
from functools import reduce 

In [2]:
re_notsymb = re.compile(r'\W+')
re_number0=re.compile(r'\d')
re_endword=re.compile(r'\w*\.')
re_space=re.compile(r'\s+')
re_colon=re.compile(r'.*:')

In [3]:
author_id={
  "Достоевский Ф. М.": 9150,
  "Роллинс Дж.": 59396,
  "Фицджеральд Ф. С.": 28727,
  "Глуховский Д. А.": 53427,
  "Стругацкий А. Н.": 26268,
  "Лукьяненко С. В.": 16626,
  "Фрай М.": 28927,
  "Хантер Э.": 37969,
  "Роулинг Дж. К.": 104832
}

In [4]:
def loger(func):
    import functools
    @functools.wraps(func)
    def wraper(*argc,**argv):
        res = func(*argc,**argv)
        with mutex:
            global n_processed
            n_processed += 1
            if n_processed % 10 == 0:
                print(f"\r{n_processed} objects are processed...", end='', flush=True)
        return res
    return wraper

In [9]:
@loger
def extract_book_info(book_id):    
    url='https://www.moscowbooks.ru'+book_id[0]
    html=requests.get(url).text
    soup=BeautifulSoup(html,'lxml')

    ans=dict()

    ans['ID']=book_id[0][6:-1]
    ans['Автор']=soup.find(class_="page-header__author").text[1:-1]
    ans['Название']=book_id[1]

    book_body=soup.find('div',class_="book__body")
    book_intro=book_body.find('div',class_="book__intro")
    book_info=book_body.find('div',class_="book__info")

    ans['Наличие']=book_intro.find(class_="book__shop-instock") is None
    tmp=book_intro.find(class_="book__price")
    ans['Цена']=''.join(re_number0.findall(tmp.text))+' '+re_endword.search(tmp.text)[0]

    description = book_info.find(class_="book__description")
    for d in description.find_all(['b','a']):
        d.replace_with('')

    for br in description.find_all("br"):
        br.replace_with("\n")

    ans['Описание']=re_space.sub(' ', description.text)[1:]



    ans['Рейтинг']=book_body.find(class_="book___rating-stars rating-stars rating-stars_lg")['data-rate']
    ans['Обложка']='https://www.moscowbooks.ru'+book_intro.find(class_="book__img book__img_default gallery__img")['src']
    ans['Стикеры']=';'.join({re_notsymb.sub('',i.text) if i.span is None else i.span['title']
                          for i in [item for j in book_intro.find_all(class_='book__stickers')
                                    for item in j.find_all(class_='stickers__item')]})






    for side in ['left','right']:
        for dl in book_info.find(class_=f'book__details-{side}').find_all('dl'):
            dt=dl.find_all('dt')
            key=re_colon.search(re_space.sub(' ', dt[0].text)[1:])[0][:-1]
            value=re_space.sub(' ', dt[1].text)[1:]
            ans[key]=value
    
    return ans

In [10]:
max_counter=5

def f(author):
    url_0 = f'https://www.moscowbooks.ru/catalog/author/{author_id[author]}/'

    downloaded=True
    counter=0

    html_0 = requests.get(url_0)
    while(html_0.status_code!=200):
        counter+=1
        if(counter==max_counter):
            downloaded=False
            break
        sleep(1)
        html_0 = requests.get(url_0)

    if not downloaded:
        return []

    soup = BeautifulSoup(html_0.text, 'lxml')
    ans=[]
    number_of_pages=max(len(soup.find_all(class_='pager__text')),2)

    for i in range(1,number_of_pages):
        url=url_0+f'?page={i}'

        downloaded=True
        counter=0
        html=requests.get(url)

        while(html.status_code!=200):
            counter+=1
            if(counter==max_counter):
                downloaded=False
                break
            sleep(1)
            html = requests.get(url)

        if not downloaded:
            continue


        soup=BeautifulSoup(html.text,'lxml')
        ans+=[(i['href'],i.text) for i in soup.find_all(class_="book-preview__title-link")]
    return ans


In [11]:
from multiprocessing.pool import ThreadPool as Pool

with Pool(9) as p:
    x=p.map(f,author_id )
x=[element for sublist in x for element in sublist]

In [12]:
from threading import Lock
import pandas as pd

mutex = Lock()
n_processed = 0

with Pool(15) as p:
    result = p.map(extract_book_info, x)
df = pd.DataFrame(result)
df.sort_values(by=['ID'], inplace=True)

with open('data/hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

240 objects are processed...